In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd drive/My\ Drive/IR/Project/

/content/drive/My Drive/IR/Project


In [ ]:
!pip install transformers[sentencepiece]
!pip install sentencepiece
!pip install datasets
!pip install rank_bm25
!pip install accelerate
!pip install -U bertopic
!pip install -U safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 34.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2

In [ ]:
# !pip install tensorflow==2.14.0

In [ ]:
import torch
import json
import re

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModel
from rank_bm25 import BM25Okapi

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, \
    Seq2SeqTrainer
from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
# from bertopic import BERTopic
import json
import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(f'Device: {device}')

Device: cpu


#Models

In [ ]:
contriever_tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
contriever_model = AutoModel.from_pretrained('facebook/contriever')

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# topic_model = BERTopic.load("MaartenGr/BERTopic_ArXiv")

#Prepare BM25

In [ ]:
def similarity_bm25(sent1, sent2):
    corpus = [sent2]
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = sent1.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)

    return abs(doc_scores[0])


def get_input_prompt_citation_identification_bm25(ques, profile, k):
    rank = []

    for x in profile:
        rank.append((similarity_bm25(x['title'], ques), x['title']))

    rank.sort(reverse=True)
    history = None

    for i, x in enumerate(rank):
        if i == k:
            break
        if i == 0:
            history = '\"' + x[1] + '\"'
        else:
            history = history + ', \"' + x[1] + '\"'

    try:
        pattern = ', which reference is related?'
        updated_prompt = ques.replace(pattern, f", {history}, which reference is related?")
        #print(updated_prompt)
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

In [ ]:
def get_nws_title(news):
  x = ''
  lst = news.split(' ')
  ok = 0

  for t in lst:
    if t.strip() == 'article:':
      ok = 1
      continue
    if ok:
      x = x + ' ' + t

  x = x.strip()
  return x



def get_input_prompt_nws_cat_bm25(ques, profile, k):
    rank = []

    nes_title = get_nws_title(ques)

    for x in profile:
        rank.append((similarity_bm25(x['text'], nes_title), x['text'], x['category']))

    rank.sort(reverse=True)
    history = None

    for i, x in enumerate(rank):
        if i == k:
            break
        if i == 0:
            history = '\"' + f"The category for the article: \"{x[1]}\" is {x[2]}." + '\"'
        else:
            history = history + ', \"' + f"The category for the article: \"{x[1]}\" is {x[2]}." + '\"'

    try:
        updated_prompt = history + ' ' + ques
        #print(updated_prompt)
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

# Contriever Rerank

In [ ]:
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def get_input_prompt_citation_identification_contriever(ques, profile, k):
    rank = []

    all_sentence = []
    sentences = []
    sentences.append(ques)

    for x in profile:
        rank.append((similarity_bm25(x['title'], ques), x['title']))

    rank.sort(reverse=True)


    for i in range(0, 10):
        sentences.append(rank[i][1])

    inputs = contriever_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    embeddings = contriever_model(**inputs)
    sentence_embeddings = mean_pooling(embeddings[0], inputs['attention_mask'])

    history = None
    rank = []

    for i in range(1, 11):
        rank.append((sentence_embeddings[0] @ sentence_embeddings[i], sentences[i]))

    rank.sort(reverse=True)

    for i, x in enumerate(rank):
        if i == k:
          break
        if i == 0:
            history = '\"' + x[1] + '\"'
        else:
            history = history + ', \"' + x[1] + '\"'

    try:
        pattern = ', which reference is related?'
        updated_prompt = ques.replace(pattern, f", and the have published before in the following topics: {history}, which reference is related?")
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

In [ ]:
def get_input_prompt_nws_cat_contriever(ques, profile, k):
    rank = []
    nws_title = get_nws_title(ques)
    all_sentence = []
    sentences = []
    sentences.append(nws_title)

    for x in profile:
        rank.append((similarity_bm25(x['text'], nws_title), x['text'], x['category']))

    rank.sort(reverse=True)

    k_first = 10
    if len(rank) < k_first:
      k_first = len(rank)

    for i in range(0, k_first):
        # print(rank[i])
        sentences.append(rank[i][1])

    inputs = contriever_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    embeddings = contriever_model(**inputs)
    sentence_embeddings = mean_pooling(embeddings[0], inputs['attention_mask'])

    history = None
    rank1 = []

    for i in range(1, k_first+1):
        rank1.append((sentence_embeddings[0] @ sentence_embeddings[i], sentences[i], rank[i-1][2]))

    rank1.sort(reverse=True)

    for i, x in enumerate(rank1):
        if i == k:
          break
        if i == 0:
            history = '\"' + f"The category for the article: \"{x[1]}\" is {x[2]}." + '\"'
        else:
            history = history + ', \"' + f"The category for the article: \"{x[1]}\" is {x[2]}." + '\"'

    try:
        updated_prompt = history + ' ' + ques
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

#Topic Model

In [ ]:
def get_relevant_topic(abstract):
    topic, prob = topic_model.transform(abstract)
    all_topics=topic_model.get_topic_info(topic[0])['Representation'][0]
    ret = ''
    for i,x in enumerate(all_topics):
      if i == 0:
        ret = x.strip()
      else:
        ret = ret + ', ' + x.strip()
    return ret

def get_input_prompt_citation_identification_topicmodel(ques, profile, k):
    rank = []

    for x in profile:
        rank.append((similarity_bm25(x['title'], ques), x['abstract']))

    rank.sort(reverse=True)

    history = None

    for i, x in enumerate(rank):
        if i == k:
          break
        if i == 0:
            history = '\"' + get_relevant_topic(x[1]) + '\"'
        else:
            history = history + ', \"' + get_relevant_topic(x[1]) + '\"'

    try:
        pattern = ', which reference is related?'
        updated_prompt = ques.replace(pattern, f", and the have published before in the following topics: {history}, which reference is related?")
        #print(updated_prompt)
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

In [ ]:
def get_input_prompt_nws_cat_topicmodel(ques, profile, k):
    rank = []
    nws_title = get_nws_title(ques)
    for x in profile:
        rank.append((similarity_bm25(x['text'], nws_title), x['text'], x['category']))

    rank.sort(reverse=True)

    history = None

    for i, x in enumerate(rank):
        if i == k:
          break
        if i == 0:
            history = '\"' + f"The category of an article with topics: {get_relevant_topic(x[1])}, text: \"{x[1]}\" is {x[2]}" + '\"'
        else:
            history = history + ', \"' + f"The category of an article with topics: {get_relevant_topic(x[1])}, text: \"{x[1]}\" is {x[2]}" + '\"'

    try:
        updated_prompt = history + ' ' + ques
        #print(updated_prompt)
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

# Clustering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
import random

random.seed(random.randint(1, 100))

def get_cluster_points(sentences, num_clusters=2):
  vectorizer = TfidfVectorizer(stop_words='english')
  vectorized_documents = vectorizer.fit_transform(sentences)
  pca = PCA(n_components=2)
  reduced_data = pca.fit_transform(vectorized_documents.toarray())
  kmeans = KMeans(n_clusters=num_clusters, n_init=5,
                  max_iter=500, random_state=42)
  kmeans.fit(vectorized_documents)
  r = {}

  for i in range(num_clusters):
    r[i] = []

  for i, x in enumerate(sentences):
    label = int(kmeans.labels_[i])
    r[label].append(x)
  res = []

  for i in range(num_clusters):
    lst_len = len(r[i])
    x = random.randint(0, lst_len-1)

    res.append(r[i][x])

  return res



In [ ]:
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def get_input_prompt_citation_identification_clustering(ques, profile, k):
    rank = []

    all_sentence = []
    for x in profile:
      all_sentence.append(x['title'])

    ll = get_cluster_points(all_sentence, 10)
    sentences = []
    sentences.append(ques)

    for x in ll:
      sentences.append(x)



    inputs = contriever_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    embeddings = contriever_model(**inputs)
    sentence_embeddings = mean_pooling(embeddings[0], inputs['attention_mask'])

    history = None
    rank = []

    for i in range(1, 11):
        rank.append((sentence_embeddings[0] @ sentence_embeddings[i], sentences[i]))

    rank.sort(reverse=True)

    for i, x in enumerate(rank):
        if i == k:
          break
        if i == 0:
            history = '\"' + x[1] + '\"'
        else:
            history = history + ', \"' + x[1] + '\"'

    try:
        pattern = ', which reference is related?'
        updated_prompt = ques.replace(pattern, f", {history}, which reference is related?")
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

In [ ]:
def get_input_prompt_nws_cat_clustering(ques, profile, k):
    rank = []
    nws_title = get_nws_title(ques)
    dd = {}

    all_sentence = []
    for x in profile:
      all_sentence.append(x['text'])
      dd[x['text']] = x['category']

    ll = get_cluster_points(all_sentence, 6)
    sentences = []
    sentences.append(nws_title)

    for x in ll:
      sentences.append(x)

    inputs = contriever_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    embeddings = contriever_model(**inputs)
    sentence_embeddings = mean_pooling(embeddings[0], inputs['attention_mask'])

    for i in range(1, 7):
        rank.append((sentence_embeddings[0] @ sentence_embeddings[i], sentences[i], dd[sentences[i]]))


    rank.sort(reverse=True)

    history = None

    for i, x in enumerate(rank):
        if i == k:
          break
        if i == 0:
            history = '\"' + f"The category for the article: \"{x[1]}\" is {x[2]}." + '\"'
        else:
            history = history + ', \"' + f"The category for the article: \"{x[1]}\" is {x[2]}." + '\"'

    try:
        updated_prompt = history + ' ' + ques
        #print(updated_prompt)
        #updated_prompt = re.sub(pattern, f", {history}, which reference is related?", ques)
        return updated_prompt
    except Exception as e:
        print(str(e))
        print(ques)
        return ques

#Load Data

In [ ]:
def load_and_merge_files_citation_identification(question_file, output_file, k=1, retriever=2):
    with open(question_file, 'r', encoding='utf-8') as file:
        questions = json.load(file)
    with open(output_file, 'r', encoding='utf-8') as file:
        outputs = json.load(file)

    inputs = []
    targets = []
    for question in questions:
        profile = question['profile']

        if retriever == 0:
          input_prompt = get_input_prompt_citation_identification_bm25(question['input'], profile, k)
        elif retriever == 1:
          input_prompt = get_input_prompt_citation_identification_topicmodel(question['input'], profile, k)
        elif retriever == 2:
          input_prompt = get_input_prompt_citation_identification_contriever(question['input'], profile, k)
        elif retriever == 3:
          input_prompt = get_input_prompt_citation_identification_clustering(question['input'], profile, k)

        output = next((out for out in outputs['golds'] if out['id'] == question['id']), None)
        if output:
            inputs.append(input_prompt)
            targets.append(output['output'])

    #print(inputs)
    #print(targets)

    raw_data = {}
    raw_data['input'] = inputs
    raw_data['target'] = targets

    return raw_data

In [ ]:
def load_and_merge_files_nws_cat(question_file, output_file, k=1, retriever=2):
    with open(question_file, 'r', encoding='utf-8') as file:
        questions = json.load(file)
    with open(output_file, 'r', encoding='utf-8') as file:
        outputs = json.load(file)

    inputs = []
    targets = []
    for question in questions:
        profile = question['profile']

        if retriever == 0:
          input_prompt = get_input_prompt_nws_cat_bm25(question['input'], profile, k)
        elif retriever == 1:
          input_prompt = get_input_prompt_nws_cat_topicmodel(question['input'], profile, k)
        elif retriever == 2:
          input_prompt = get_input_prompt_nws_cat_contriever(question['input'], profile, k)
        elif retriever == 3:
          input_prompt = get_input_prompt_nws_cat_clustering(question['input'], profile, k)

        output = next((out for out in outputs['golds'] if out['id'] == question['id']), None)
        if output:
            inputs.append(input_prompt)
            targets.append(output['output'])

    #print(inputs)
    #print(targets)

    raw_data = {}
    raw_data['input'] = inputs
    raw_data['target'] = targets

    return raw_data

In [ ]:
def get_dataset(k, type=0, retriever=2):
    if type == 0:
      train_data_path = (
      'data/citation_identification/train_questions.json', 'data/citation_identification/train_outputs.json')
      dev_data_path = (
      'data/citation_identification/dev_questions.json', 'data/citation_identification/dev_outputs.json')
    elif type == 1:
      train_data_path = (
      'data/time_citation_identification/train_questions.json', 'data/time_citation_identification/train_outputs.json')
      dev_data_path = (
      'data/time_citation_identification/dev_questions.json', 'data/time_citation_identification/dev_outputs.json')


    raw_data_train = load_and_merge_files_citation_identification(train_data_path[0], train_data_path[1], k, retriever)
    raw_data_dev = load_and_merge_files_citation_identification(dev_data_path[0], dev_data_path[1], k, retriever)


    return raw_data_train, raw_data_dev


def get_dataset_nws_cat(k, type=0, retriever=2):
    if type == 0:
      train_data_path = (
      'data/news_categorization/train_questions.json', 'data/news_categorization/train_outputs.json')
      dev_data_path = (
      'data/news_categorization/dev_questions.json', 'data/news_categorization/dev_outputs.json')
    elif type == 1:
      train_data_path = (
      'data/time_news_categorization/train_questions.json', 'data/time_news_categorization/train_outputs.json')
      dev_data_path = (
      'data/time_news_categorization/dev_questions.json', 'data/time_news_categorization/dev_outputs.json')


    raw_data_train = load_and_merge_files_nws_cat(train_data_path[0], train_data_path[1], k, retriever)
    raw_data_dev = load_and_merge_files_nws_cat(dev_data_path[0], dev_data_path[1], k, retriever)


    return raw_data_train, raw_data_dev

In [ ]:
raw_data_train, raw_data_dev = get_dataset_nws_cat(k=1, type=0, retriever=3)

if not os.path.exists('data/news_categorization/raw/cluster/k_1'):
    os.makedirs('data/news_categorization/raw/cluster/k_1')

with open('data/news_categorization/raw/cluster/k_1/train.json', 'w') as fp:
    json.dump(raw_data_train, fp)


with open('data/news_categorization/raw/cluster/k_1/dev.json', 'w') as fp:
    json.dump(raw_data_dev, fp)

In [ ]:
raw_data_train, raw_data_dev = get_dataset_nws_cat(k=4, type=0, retriever=3)

if not os.path.exists('data/news_categorization/raw/cluster/k_4'):
    os.makedirs('data/news_categorization/raw/cluster/k_4')

with open('data/news_categorization/raw/cluster/k_4/train.json', 'w') as fp:
    json.dump(raw_data_train, fp)


with open('data/news_categorization/raw/cluster/k_4/dev.json', 'w') as fp:
    json.dump(raw_data_dev, fp)

In [ ]:
raw_data_train, raw_data_dev = get_dataset_nws_cat(k=1, type=1, retriever=3)

if not os.path.exists('data/time_news_categorization/raw/cluster/k_1'):
    os.makedirs('data/time_news_categorization/raw/cluster/k_1')

with open('data/time_news_categorization/raw/cluster/k_1/train.json', 'w') as fp:
    json.dump(raw_data_train, fp)


with open('data/time_news_categorization/raw/cluster/k_1/dev.json', 'w') as fp:
    json.dump(raw_data_dev, fp)

In [ ]:
raw_data_train, raw_data_dev = get_dataset_nws_cat(k=4, type=1, retriever=3)

if not os.path.exists('data/time_news_categorization/raw/cluster/k_4'):
    os.makedirs('data/time_news_categorization/raw/cluster/k_4')

with open('data/time_news_categorization/raw/cluster/k_4/train.json', 'w') as fp:
    json.dump(raw_data_train, fp)


with open('data/time_news_categorization/raw/cluster/k_4/dev.json', 'w') as fp:
    json.dump(raw_data_dev, fp)

#Read/Write JSON File

In [ ]:
def read_json_dict(filename: str):
    with open(filename) as fp:
        return json.load(fp)

In [ ]:
#Model 1
user_citation_bm25_k_1_train = read_json_dict('data/news_categorization/raw/bm25/k_1/train.json')
user_citation_bm25_k_1_dev =  read_json_dict('data/news_categorization/raw/bm25/k_1/dev.json')

#Model 2
user_citation_bm25_k_4_train = read_json_dict('data/news_categorization/raw/bm25/k_4/train.json')
user_citation_bm25_k_4_dev = read_json_dict('data/news_categorization/raw/bm25/k_4/dev.json')

#Model 3
user_citation_topic_model_k_1_train = read_json_dict('data/news_categorization/raw/topic_model/k_1/train.json')
user_citation_topic_model_k_1_dev = read_json_dict('data/news_categorization/raw/topic_model/k_1/dev.json')

#Model 4
user_citation_topic_model_k_4_train = read_json_dict('data/news_categorization/raw/topic_model/k_4/train.json')
user_citation_topic_model_k_4_dev = read_json_dict('data/news_categorization/raw/topic_model/k_4/dev.json')

#Model 5
time_citation_bm25_k_1_train = read_json_dict('data/time_news_categorization/raw/bm25/k_1/train.json')
time_citation_bm25_k_1_dev = read_json_dict('data/time_news_categorization/raw/bm25/k_1/dev.json')

#Model 6
time_citation_bm25_k_4_train = read_json_dict('data/time_news_categorization/raw/bm25/k_4/train.json')
time_citation_bm25_k_4_dev = read_json_dict('data/time_news_categorization/raw/bm25/k_4/dev.json')

#Model 7
time_citation_topic_model_k_1_train = read_json_dict('data/time_news_categorization/raw/topic_model/k_1/train.json')
time_citation_topic_model_k_1_dev = read_json_dict('data/time_news_categorization/raw/topic_model/k_1/dev.json')

#Model 8
time_citation_topic_model_k_4_train = read_json_dict('data/time_news_categorization/raw/topic_model/k_4/train.json')
time_citation_topic_model_k_4_dev = read_json_dict('data/time_news_categorization/raw/topic_model/k_4/dev.json')


#Model 9
user_citation_rerank_cont_k_1_train = read_json_dict('data/news_categorization/raw/rerank_cont/k_1/train.json')
user_citation_rerank_cont_k_1_dev =  read_json_dict('data/news_categorization/raw/rerank_cont/k_1/dev.json')

#Model 10
user_citation_rerank_cont_k_4_train = read_json_dict('data/news_categorization/raw/rerank_cont/k_4/train.json')
user_citation_rerank_cont_k_4_dev =  read_json_dict('data/news_categorization/raw/rerank_cont/k_4/dev.json')

#Model 11
time_citation_rerank_cont_k_1_train = read_json_dict('data/time_news_categorization/raw/rerank_cont/k_1/train.json')
time_citation_rerank_cont_k_1_dev =  read_json_dict('data/time_news_categorization/raw/rerank_cont/k_1/dev.json')

#Model 12
time_citation_rerank_cont_k_4_train = read_json_dict('data/time_news_categorization/raw/rerank_cont/k_4/train.json')
time_citation_rerank_cont_k_4_dev =  read_json_dict('data/time_news_categorization/raw/rerank_cont/k_4/dev.json')